<a href="https://colab.research.google.com/github/blackleg-sanji007/Traffic-Safety-and-Description-Analysis-of-WTS-Dataset/blob/main/Caption_analysis_using_LLm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the zip file which contains data of captions.

In [ ]:
import zipfile

zip_path = '/content/train_zip.zip'

extract_path = 'CE_data1'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Code to extract data (captions in our case) which is stored in json fromat. We have taken only overhead view in account. We have taken Label 4 (last labels) from captions to keep the project computationally feesable.

In [ ]:
import os
import json

def extract_data(data_path):
    for i in os.listdir(data_path):
        i_path = os.path.join(data_path, i)
        for j in os.listdir(i_path):
            if j == "overhead_view": # we are only trying to analyse vehicle view captions.
                continue
            j_path = os.path.join(i_path, j)
            for k in os.listdir(j_path):
                k_path = os.path.join(j_path, k)

                # Read the JSON file
                if k_path.endswith('.json'):
                    with open(k_path, 'r') as f:
                        data = json.load(f)

                    print("\n=== Vehicle View:", data.get("vehicle_view", "Unknown"), "===\n")

                    # for idx, event in enumerate(data.get("event_phase", [])):
                    #     cap_ped = event.get("caption_pedestrian", "No caption_pedestrian")
                    #     cap_veh = event.get("caption_vehicle", "No caption_vehicle")
                    #     print(f"Event {idx+1}:")
                    #     print("Pedestrian Caption:\n", cap_ped)
                    #     print("Vehicle Caption:\n", cap_veh)
                    #     print("\n---\n")

                    event_phase = data.get("event_phase", [])

                    if event_phase:  # Check if the list is not empty
                        event = event_phase[0]  # Get only the first event

                        cap_ped = event.get("caption_pedestrian", "No caption_pedestrian")
                        cap_veh = event.get("caption_vehicle", "No caption_vehicle")

                        print("Event 1:")
                        print("Pedestrian Caption:\n", cap_ped)
                        print("Vehicle Caption:\n", cap_veh)
                        print("\n---\n")
                    else:
                        print("No events found.")

Here event 1 denotes label 4 (5 actually since labels are 0-index)

In [ ]:
path = '/content/CE_data1/train'
extract_data(path)


=== Vehicle View: 20230728_33_CN25_T2_vehicle_view.mp4 ===

Event 1:
Pedestrian Caption:
 The pedestrian, a man in his 30s wearing a black T-shirt and navy blue slacks, was diagonally positioned to the left of the vehicle, facing it. He stood close to the vehicle on a bright and dry road surface. With a height of 170 cm and glasses on, he appeared to be closely watching the vehicle. The pedestrian, who seemed to be aware of the vehicle but had almost noticed it, was preparing to cross diagonally. However, as he was about to make his move, a collision occurred. Despite the pedestrian's actions and direction of travel being in front of the vehicle, the collision took place due to their close proximity and the pedestrian's lack of awareness of the vehicle. The environment conditions, such as the brightness and road surface conditions being favorable, did not contribute to the collision.
Vehicle Caption:
 The vehicle was positioned diagonally to the left in front of the pedestrian and was

In this section we are converting the text obtained from json fromat to csv for easy usability.

In [ ]:
import os
import json
import csv

def extract_data_to_csv(data_path, output_csv='vehicle_view_captions1.csv'):
    rows = []

    for i in os.listdir(data_path):
        i_path = os.path.join(data_path, i)
        for j in os.listdir(i_path):
            if j == "overhead_view":
                continue
            j_path = os.path.join(i_path, j)
            for k in os.listdir(j_path):
                k_path = os.path.join(j_path, k)

                if k_path.endswith('.json'):
                    with open(k_path, 'r') as f:
                        data = json.load(f)

                    vehicle_view = data.get("vehicle_view", "Unknown")
                    pedestrian_captions = []
                    vehicle_captions = []

                    for event in data.get("event_phase", []):
                        pedestrian_captions.append(event.get("caption_pedestrian", ""))
                        vehicle_captions.append(event.get("caption_vehicle", ""))

                    rows.append({
                        'vehicle_view': vehicle_view,
                        'captions_pedestrian': "\n\n".join(pedestrian_captions),
                        'captions_vehicle': "\n\n".join(vehicle_captions)
                    })

    # Write to CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['vehicle_view', 'captions_pedestrian', 'captions_vehicle']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    print(f"✅ Data extracted and saved to {output_csv}")

In [ ]:
extract_data_to_csv(path)

✅ Data extracted and saved to vehicle_view_captions1.csv


converting the data to 'panda' dataframe for usability.

In [ ]:
import pandas as pd
data = pd.read_csv('vehicle_view_captions1.csv')
data.head()

,vehicle_view,captions_pedestrian,captions_vehicle
0,20230728_33_CN25_T2_vehicle_view.mp4,"The pedestrian, a man in his 30s wearing a bla...",The vehicle was positioned diagonally to the l...
1,20230929_46_SN38_T1_vehicle_view.mp4,"The pedestrian, a male in his 30s, was walking...",The vehicle is positioned on the right side of...
2,20230728_34_SN48_T1_vehicle_view.mp4,"The pedestrian, a male in his 30s with a heigh...",The vehicle was positioned behind the pedestri...
3,20230929_71_SN18_T1_vehicle_view.mp4,The pedestrian is a male in his 30s with a hei...,The vehicle was positioned on the left side of...
4,20230929_37_SY19_T1_vehicle_view.mp4,"The pedestrian, in his 20s and standing at a h...",The vehicle is positioned on the right side of...


Printing the data (captions).

In [ ]:
print(data.iloc[1,2])

The vehicle is positioned on the left side of the pedestrian and is relatively close to them. The pedestrian is visible within the vehicle's field of view. Suddenly, the vehicle collides with the pedestrian as it is moving at a speed of 10 km/h. In regards to the environment conditions, the pedestrian is a male in his 30s with a height of 170 cm. He is wearing a black T-shirt as his upper body clothing and gray slacks for his lower body clothing. The incline of the road surface where the event takes place is level, and it is made of asphalt.

The vehicle is positioned diagonally to the left in front of the pedestrian, close in distance. The vehicle's field of view allows it to see the pedestrian. It is proceeding straight ahead at a speed of 5 km/h. The environment conditions reveal that the pedestrian is a male in his 30s with a height of 170 cm. He is wearing a black T-shirt on his upper body and gray slacks on his lower body. The road surface is level and made of asphalt.

The vehic

We have used Openai-python code. For connectiong our codebase to LLM model 'LaLma 3.70B' via API by Using the OpenAI SDK.

In [ ]:
!pip install openai

from openai import OpenAI

Creating 'Analysis' function which sends our request to LLM mode using API key. The following code is standered code use for conncting our query to LLM via use of API-key.

Here base-url is URL of our LLM provider 'OpernRouter' which then forwards our request to LaLma 3.70B model. To put it simply it is host serevr of LLM.This means we are using OpenRouter, which is an API platform compatible with the OpenAI API format, but it serves models from multiple providers (like Meta’s LLaMA, Mistral, etc.)


In [ ]:

def analysis(caption):
          client = OpenAI(
            base_url="https://openrouter.ai/api/v1", #
            api_key="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",
          )
          completion = client.chat.completions.create(
            extra_headers={
              "HTTP-Referer": "<YOUR_SITE_URL>", # Site URL, incase if user wants to integrate this analysis model to a website
              "X-Title": "<YOUR_SITE_NAME>", # Name of site.
            },
            extra_body={},
            model="meta-llama/llama-3.3-70b-instruct:free", #model used for caption analysis
            messages=[
              {
                "role": "user", # tells or defines the identity of enetiy askiing the query. Who is speaking in the conversation.
                "content": f"How much is the following caption relevant from a traffic engineering perspective:\n\n{caption}\n\n" # promps (Query) provied to LLM. This is the actual text/message that the 'role' is providing.
              }
            ]
          )
          print(completion.choices[0].message.content)
          return completion.choices[0].message.content #returning the obtained output.

Storing the obtained anlysis in form of an array.

In [ ]:
arr = []
for i in data['captions_vehicle']:
  arr.append(analysis(i))

From a traffic engineering perspective, the relevance of the provided captions can be assessed based on the information they contain regarding road design, traffic conditions, vehicle and pedestrian behavior, and environmental factors that could influence traffic safety and operations. Here's a breakdown of the relevance of each caption:

1. **First Caption**: This caption is relevant as it provides details about the vehicle's position relative to the pedestrian, the vehicle's speed (10 km/h), and the road conditions (dry, level, asphalt, two-way traffic residential road). The mention of no sidewalks and the presence of street lights is also relevant. However, details about the driver's and pedestrian's attire are not relevant from a traffic engineering perspective.

2. **Second Caption**: This caption discusses a scenario where the vehicle is stationary (0 km/h) and collides with a pedestrian. The details about the accident scenario, road conditions (dry, level, residential road), and

KeyboardInterrupt: 